In [1]:
import xml.etree.ElementTree as ET
from datetime import datetime

class MetadataExtractor:
    """Clase base para la extracción de metadatos"""
    
    def __init__(self):
        self.namespaces = {
            'gmd': 'http://www.isotc211.org/2005/gmd',
            'gco': 'http://www.isotc211.org/2005/gco',
            'gts': 'http://www.isotc211.org/2005/gts',
            'gml': 'http://www.opengis.net/gml/3.2',
            'xsi': 'http://www.w3.org/2001/XMLSchema-instance'
        }

    def get_element_text(self, element, xpath):
        """Extrae el texto de un elemento XML"""
        elem = element.find(xpath, self.namespaces)
        if elem is not None:
            return elem.text
        return None

    def get_code_list_value(self, element, xpath):
        """Obtiene el valor de una lista de códigos"""
        elem = element.find(xpath, self.namespaces)
        if elem is not None:
            return elem.get('codeListValue')
        return None

    def extract_common_metadata(self, root):
        """Extrae los metadatos comunes para todos los tipos"""
        metadata = {
            'metadato': {
                'codigo': self.get_element_text(root, './/gmd:fileIdentifier/gco:CharacterString'),
                'idioma': self.get_element_text(root, './/gmd:language/gco:CharacterString'),
                'codificacion': self.get_code_list_value(root, './/gmd:characterSet/gmd:MD_CharacterSetCode'),
                'contacto': self.extract_contact_info(root.find('.//gmd:contact/gmd:CI_ResponsibleParty', self.namespaces)),
                'fecha': self.extract_date_info(root.find('.//gmd:dateStamp', self.namespaces)),
                'norma': {
                    'nombre': self.get_element_text(root, './/gmd:metadataStandardName/gco:CharacterString'),
                    'version': self.get_element_text(root, './/gmd:metadataStandardVersion/gco:CharacterString')
                },
                'perfil': {
                    'nombre': self.get_element_text(root, './/gmd:metadataProfileName/gco:CharacterString'),
                    'version': None  # Opcional
                },
                'ambito': self.get_code_list_value(root, './/gmd:hierarchyLevel/gmd:MD_ScopeCode')
            },
            'identificacion': self.extract_identification_info(root)
        }
        return metadata

    def extract_contact_info(self, contact_element):
        """Extrae la información de contacto"""
        if contact_element is None:
            return None
            
        return {
            'rol': self.get_code_list_value(contact_element, './/gmd:role/gmd:CI_RoleCode'),
            'organizacion': self.get_element_text(contact_element, './/gmd:organisationName/gco:CharacterString'),
            'cargo': self.get_element_text(contact_element, './/gmd:positionName/gco:CharacterString'),
            'telefono': {
                'numero': self.get_element_text(contact_element, './/gmd:voice/gco:CharacterString'),
                'tipo': self.get_code_list_value(contact_element, './/gmd:CI_TelephoneTypeCode')
            },
            'direccion': self.get_element_text(contact_element, './/gmd:deliveryPoint/gco:CharacterString'),
            'ciudad': self.get_element_text(contact_element, './/gmd:city/gco:CharacterString'),
            'pais': self.get_element_text(contact_element, './/gmd:country/gco:CharacterString'),
            'email': self.get_element_text(contact_element, './/gmd:electronicMailAddress/gco:CharacterString'),
            'enlace': self.get_element_text(contact_element, './/gmd:linkage/gmd:URL')
        }

    def extract_date_info(self, date_element):
        """Extrae la información de fecha"""
        if date_element is None:
            return None
            
        return {
            'valor': self.get_element_text(date_element, './/gco:DateTime') or 
                    self.get_element_text(date_element, './/gco:Date'),
            'tipo': self.get_code_list_value(date_element, './/gmd:CI_DateTypeCode')
        }

    def extract_identification_info(self, root):
        """Extrae la información de identificación común"""
        ident = root.find('.//gmd:identificationInfo/gmd:MD_DataIdentification', self.namespaces)
        if ident is None:
            return None

        extent = ident.find('.//gmd:extent//gmd:EX_GeographicBoundingBox', self.namespaces)
        return {
            'titulo': self.get_element_text(ident, './/gmd:citation//gmd:title/gco:CharacterString'),
            'fecha': self.extract_date_info(ident.find('.//gmd:citation//gmd:date', self.namespaces)),
            'resumen': self.get_element_text(ident, './/gmd:abstract/gco:CharacterString'),
            'contacto': self.extract_contact_info(ident.find('.//gmd:pointOfContact/gmd:CI_ResponsibleParty', self.namespaces)),
            'tipo_representacion': self.get_code_list_value(ident, './/gmd:spatialRepresentationType/gmd:MD_SpatialRepresentationTypeCode'),
            'categoria_tema': self.get_code_list_value(ident, './/gmd:topicCategory/gmd:MD_TopicCategoryCode'),
            'extension': self.extract_extent_info(extent),
            'mantenimiento': {
                'frecuencia': self.get_code_list_value(ident, './/gmd:resourceMaintenance//gmd:maintenanceAndUpdateFrequency/gmd:MD_MaintenanceFrequencyCode')
            },
            'palabras_clave': self.extract_keywords(ident)
        }

    def extract_extent_info(self, extent):
        """Extrae la información de extensión geográfica"""
        if extent is None:
            return None
            
        return {
            'oeste': self.get_element_text(extent, './/gmd:westBoundLongitude/gco:Decimal'),
            'este': self.get_element_text(extent, './/gmd:eastBoundLongitude/gco:Decimal'),
            'sur': self.get_element_text(extent, './/gmd:southBoundLatitude/gco:Decimal'),
            'norte': self.get_element_text(extent, './/gmd:northBoundLatitude/gco:Decimal')
        }

    def extract_keywords(self, ident):
        """Extrae las palabras clave"""
        keywords = []
        for keyword in ident.findall('.//gmd:keyword/gco:CharacterString', self.namespaces):
            if keyword.text:
                keywords.append(keyword.text)
        return keywords

class VectorMetadataExtractor(MetadataExtractor):
    """Extractor específico para metadatos de tipo vector"""

    def extract(self, root):
        metadata = self.extract_common_metadata(root)
        vector_specific = {
            'vector_info': {
                'nivel_topologia': self.get_element_text(root, './/gmd:topologyLevel/gco:CharacterString'),
                'objetos_geometricos': self.extract_geometric_objects(root),
                'denominador_escala': self.get_element_text(root, './/gmd:equivalentScale/gmd:MD_RepresentativeFraction/gmd:denominator/gco:Integer')
            }
        }
        metadata.update(vector_specific)
        return metadata

    def extract_geometric_objects(self, root):
        """Extrae información sobre objetos geométricos"""
        objects = []
        for obj in root.findall('.//gmd:geometricObjects/gmd:MD_GeometricObjects', self.namespaces):
            objects.append({
                'tipo': self.get_code_list_value(obj, './/gmd:geometricObjectType/gmd:MD_GeometricObjectTypeCode'),
                'cantidad': self.get_element_text(obj, './/gmd:geometricObjectCount/gco:Integer')
            })
        return objects

class RasterMetadataExtractor(MetadataExtractor):
    """Extractor específico para metadatos de tipo raster"""

    def extract(self, root):
        metadata = self.extract_common_metadata(root)
        raster_specific = {
            'raster_info': {
                'numero_dimensiones': self.get_element_text(root, './/gmd:numberOfDimensions/gco:Integer'),
                'dimensiones': self.extract_dimensions(root),
                'geometria_celda': self.get_code_list_value(root, './/gmd:cellGeometry/gmd:MD_CellGeometryCode'),
                'imagen': self.extract_image_description(root)
            }
        }
        metadata.update(raster_specific)
        return metadata

    def extract_dimensions(self, root):
        """Extrae información sobre las dimensiones"""
        dimensions = []
        for dim in root.findall('.//gmd:axisDimensionProperties/gmd:MD_Dimension', self.namespaces):
            dimensions.append({
                'nombre': self.get_code_list_value(dim, './/gmd:dimensionName/gmd:MD_DimensionNameTypeCode'),
                'tamaño': self.get_element_text(dim, './/gmd:dimensionSize/gco:Integer'),
                'resolucion': self.get_element_text(dim, './/gmd:resolution/gco:Measure')
            })
        return dimensions

    def extract_image_description(self, root):
        """Extrae descripción de la imagen"""
        img = root.find('.//gmd:MD_ImageDescription', self.namespaces)
        if img is None:
            return None
            
        return {
            'iluminacion': self.get_element_text(img, './/gmd:illuminationElevationAngle/gco:Real'),
            'condicion': self.get_code_list_value(img, './/gmd:imagingCondition/gmd:MD_ImagingConditionCode'),
            'calidad': self.get_element_text(img, './/gmd:imageQualityCode/gco:CharacterString'),
            'porcentaje_nubes': self.get_element_text(img, './/gmd:cloudCoverPercentage/gco:Real')
        }

def main():
    # Ejemplo de uso
    xml_file = "./metadatos/vector/test0.xml"
    # xml_file = "./metadatos/raster/test1.xml"
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    # Determinar el tipo de datos
    spatial_type = root.find('.//gmd:spatialRepresentationType/gmd:MD_SpatialRepresentationTypeCode', 
                           {'gmd': 'http://www.isotc211.org/2005/gmd'})
    
    if spatial_type is not None:
        type_value = spatial_type.get('codeListValue')
        
        if type_value == 'vector':
            extractor = VectorMetadataExtractor()
        elif type_value == 'grid':
            extractor = RasterMetadataExtractor()
        else:
            raise ValueError(f"Tipo de datos no soportado: {type_value}")
            
        metadata = extractor.extract(root)
        
        # Imprimir resultados
        print("Metadatos extraídos:")
        print_metadata(metadata)

def print_metadata(metadata, indent=0):
    """Imprime los metadatos de forma estructurada"""
    for key, value in metadata.items():
        if isinstance(value, dict):
            print("  " * indent + f"{key}:")
            print_metadata(value, indent + 1)
        else:
            print("  " * indent + f"{key}: {value}")

if __name__ == "__main__":
    main()

Metadatos extraídos:
metadato:
  codigo: 180a76a8-6e88-488d-9702-02046d9c957b
  idioma: spa
  codificacion: utf8
  contacto:
    rol: author
    organizacion: Instituto de Hidrología, Meteorología y Estudios Ambientales - IDEAM- Subdirección de Estudios Ambientales- Grupo de Seguimiento a la Sostenibilidad del Desarrollo
    cargo: Contratista
    telefono:
      numero: 57 1 3527160 Ext.601
      tipo: None
    direccion: Calle 25D No. 96B -70 Piso 2
    ciudad: Bogotá D.C.
    pais: Colombia
    email: almontenegro@ideam.gov.co
    enlace: www.ideam.gov.co
  fecha:
    valor: 2024-10-31T11:55:31
    tipo: None
  norma:
    nombre: ISO 19115:2003/19139
    version: 1.0
  perfil:
    nombre: None
    version: None
  ambito: dataset
identificacion:
  titulo: Instalaciones autorizadas para el almacenamiento, tratamiento, aprovechamiento, y/o disposición final de residuos peligrosos y/o aparatos eléctricos y electrónicos (RAEE)
  fecha:
    valor: 2024-06-20T12:00:00
    tipo: creation
  